In [1]:
import csv
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy import *
from datetime import datetime
from operator import itemgetter
from collections import defaultdict
%matplotlib inline
#plt.style.use('ggplot')
df=pd.read_excel('H:/lottery/ssq.xls',sheet_name='data')
#dfindex=pd.read_csv('C:/Users/xiang.li/Desktop/test1.txt',sep='\s+',low_memory=False)
#df['OPENID']=df['OPENID'].astype('str')
#df=pd.merge(df,dfindex,how='outer',on='pin')
df.columns=['num', 'date', 'r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'b', 'sr1', 'sr2',
       'sr3', 'sr4', 'sr5', 'sr6', 'sales_volume', 'pool', 'first', 'bonus1', 'second', 'bonus2',
       'third', 'bonus3', 'fourth', 'bonus4', 'fifth', 'bonus5', 'sexth', 'bonus6']
#df.tail()

In [ ]:
################    1.1 preprocessing data for sorted data

In [2]:
X_1 = df.iloc[:,2:8]
X_1.tail()

,r1,r2,r3,r4,r5,r6
2181,5,13,14,23,25,31
2182,1,7,10,11,26,27
2183,2,3,5,9,13,28
2184,6,15,17,18,23,30
2185,4,5,11,14,28,32


In [3]:
for i in range(1,34):
    X_1['R' + str(i)] = 0
X_1.head()
def transfer(X_1):
    for i in X_1[:6]:
        X_1[i+5] = 1
    return X_1
X = X_1.apply(transfer, axis = 1)
X = X.iloc[:, 6:]
X.tail()

'''
from sklearn.preprocessing import MultiLabelBinarizer
MultiLabelBinarizer().fit_transform(X.values)
'''

,R1,R2,R3,R4,R5,R6,R7,R8,R9,R10,...,R24,R25,R26,R27,R28,R29,R30,R31,R32,R33
2181,0,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
2182,1,0,0,0,0,0,1,0,0,1,...,0,0,1,1,0,0,0,0,0,0
2183,0,1,1,0,1,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
2184,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2185,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0


In [4]:
starttime = datetime.now()
(num_rows, num_columns) = X.shape
def creat_datasets(X, features ):
    df_ = pd.DataFrame(columns = list(range(0, features)))
    index = 0
    for i in range(features, num_rows+1):
        df_.loc[index] = X[col][i-features: i].reset_index(drop = True)
        index += 1
    return df_

for col in X.columns :
    locals()['df_'+ str(col)] = creat_datasets(X, 100)
endtime = datetime.now()
Time = endtime - starttime
print(Time.seconds, 'seconds')
# 至此33个数据集完成
# the processing take about 6 mins, so the operation of df is lower efficient than that of dict

302 seconds


In [ ]:
################    1.2 RF classifier model parameters choose and predict for sorted data

In [31]:
####    try on data df_R1
X_train = df_R1.iloc[:, 0: 99].values
y_train = df_R1[99].values
y_train = [1 if x else 0 for x in y_train]
#here Y must be boolan values? otherwize :Unknown label type: 'unknown'
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

####    gridsearchCV looking for best parameters
starttime = datetime.now()
#start = time.time()

parameter_space = {'max_features':[2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 30, 50, 99, 'auto'], 
                   'n_estimators': [20, 50, 100, 150], 
                   'criterion': ['gini', 'entropy'], 
                   'min_samples_leaf': [2, 4, 6, 8, 10, 20, 50],
                   'max_depth': [50, 100, 150, 200]}
clf = RandomForestClassifier(random_state=14, class_weight='balanced')
grid = GridSearchCV(clf, parameter_space, n_jobs=-1, scoring=None)
grid.fit(X_train, y_train)
print('Accuracy: {0:.1f}%'.format(grid.best_score_ *100))
print(grid.best_estimator_)
endtime = datetime.now()
#end = time.time()
#print(start-end)
Time = endtime - starttime
print(Time.seconds, 'seconds')

Accuracy: 80.5%
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=50, max_features=99, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=6,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=20, n_jobs=1, oob_score=False, random_state=14,
            verbose=0, warm_start=False)
1583 seconds


In [8]:
#########    RF predict result
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
starttime = datetime.now()
#start = time.time()

pipeline_RF = Pipeline([('clf', RandomForestClassifier(random_state=14, class_weight='balanced'))])
# class_weight='balanced' can solve sparse problem
parameter_space = {'clf__max_features':[2, 30, 50, 99, 'auto'], 
                   'clf__n_estimators': [10, 20, 50], 
                   'clf__criterion': ['gini', 'entropy'], 
                   'clf__min_samples_leaf': [2, 4, 6, 8, 10],
                   'clf__max_depth': [20, 50, 100]}

for i in range(1,34):
    df_ =  locals()['df_R'+ str(i)]
    X_train = df_.iloc[:, : -1].values
    y_ = df_[99].values
    y_train = [1 if x else 0 for x in y_]
    grid_RF = GridSearchCV(pipeline_RF, parameter_space, n_jobs=-1, verbose=1, scoring=None)
    grid_RF.fit(X_train, y_train)
    # evaluation for best parameters
    precision = cross_val_score(grid_RF.best_estimator_, X_train, y_train, scoring='precision')#预测为正类的样本中实际有多少是正
    recall = cross_val_score(grid_RF.best_estimator_, X_train, y_train, scoring='recall')#实际是正类的样本有多少被预测出来
    fls = cross_val_score(grid_RF.best_estimator_, X_train, y_train, scoring='f1')
    new = grid_RF.predict(df_.iloc[-1,1: ].values.reshape(1,-1))
    p_new = grid_RF.predict_proba(df_.iloc[-1,1: ].values.reshape(1,-1))[0][1]
    print('df_R'+ str(i), new, p_new, 'precison:', precision, 'recall:', recall, 'f1:', fls)
    print(grid_RF.best_estimator_)

endtime = datetime.now()
#end = time.time()
#print(start-end)
Time = endtime - starttime
print(Time.seconds, 'seconds')

Fitting 3 folds for each of 450 candidates, totalling 1350 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   20.0s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   41.9s
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 1350 out of 1350 | elapsed:  2.3min finished


df_R1 [0] 0.301629774923 precison: [ 0.66666667  0.66666667  0.5       ] recall: [ 0.01459854  0.01470588  0.00735294] f1: [ 0.02857143  0.02877698  0.01449275]
Pipeline(steps=[('clf', RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=50, max_features=2,
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=14, verbose=0, warm_start=False))])
Fitting 3 folds for each of 450 candidates, totalling 1350 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   22.3s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   46.5s
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 1350 out of 1350 | elapsed:  2.3min finished
D:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision

df_R2 [0] 0.221032267467 precison: [ 0.  0.  0.] recall: [ 0.  0.  0.] f1: [ 0.  0.  0.]
Pipeline(steps=[('clf', RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='entropy', max_depth=20, max_features='auto',
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=14, verbose=0, warm_start=False))])
Fitting 3 folds for each of 450 candidates, totalling 1350 fits


...

df_R3 [0] 0.228303110134 precison: [ 0.  0.  0.] recall: [ 0.  0.  0.] f1: [ 0.  0.  0.]
Pipeline(steps=[('clf', RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=20, max_features=30,
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=14, verbose=0, warm_start=False))])
Fitting 3 folds for each of 450 candidates, totalling 1350 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   22.4s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   45.7s
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 1350 out of 1350 | elapsed:  2.4min finished
D:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision

df_R4 [0] 0.27306951496 precison: [ 0.  0.  0.] recall: [ 0.  0.  0.] f1: [ 0.  0.  0.]
Pipeline(steps=[('clf', RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='entropy', max_depth=20, max_features='auto',
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=14, verbose=0, warm_start=False))])
Fitting 3 folds for each of 450 candidates, totalling 1350 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   23.6s


KeyboardInterrupt: 

In [ ]:
#########RF predict r_number results

'''starttime = datetime.now()
#start = time.time()

pipeline_RF = Pipeline([('clf', RandomForestClassifier(random_state=14))])
parameter_space = {'clf__max_features':[2, 30, 50, 99, 'auto'], 
                   'clf__n_estimators': [10, 20, 50], 
                   'clf__criterion': ['gini', 'entropy'], 
                   'clf__min_samples_leaf': [2, 4, 6, 8, 10],
                   'clf__max_depth': [20]}

pro_ball = defaultdict(int)
new = []
for r in range(-10, 0):
    
    for i in range(1,34):
        df_ =  locals()['df_R'+ str(i)]
        X_train = df_.iloc[:r, : -1].values
        y_train = df_[99][:r].values
        y_train = [1 if x else 0 for x in y_train]
        grid_RF = GridSearchCV(pipeline_RF, parameter_space, n_jobs=-1, verbose=1)
        grid_RF.fit(X_train, y_train)
        print('df_R'+ str(i), grid_RF.predict(df_.iloc[r-1,1: ].values.reshape(1,-1)), 
              grid_RF.predict_proba(df_.iloc[r-1,1: ].values.reshape(1,-1)))
        print(grid_RF.best_estimator_)
        pro_ball[i] = grid_RF.predict_proba(df_.iloc[r-1, 1: ].values.reshape(1, -1))
    new.append([k for (k, v2) in sorted({k: v2 for k, [[ v1,  v2]] in pro_ball.items()}.items(), key=itemgetter(1), reverse=True)[:6]])

endtime = datetime.now()
#end = time.time()
#print(start-end)
Time = endtime - starttime
print(Time.seconds, 'seconds')'''

In [ ]:
################    2.1 DT classifier model parameters choose and predict

In [10]:
#########    DT   predict result
#gridsearchCV looking for best parameters
starttime = datetime.now()
#start = time.time()

pipeline_DT = Pipeline([('clf', DecisionTreeClassifier(random_state=14, class_weight='balanced'))])
paras_DT = {'clf__max_depth': (10, 20, 30, 40),
            'clf__criterion': ('gini', 'entropy'),
            'clf__min_samples_split': (2, 3, 5),
            'clf__min_samples_leaf': (20, 30, 40)}

for i in range(1,34):
    df_ =  locals()['df_R'+ str(i)]
    X_train = df_.iloc[:, : -1].values
    y_ = df_[99].values
    y_train = [1 if x else 0 for x in y_]
    grid_DT=GridSearchCV(pipeline_DT, paras_DT, n_jobs=-1, verbose=1, scoring=None)
    #default=None, and the default scorer of DT is accuracy. see also DecisionTreeClassifier.score(X, y)
    grid_DT.fit(X_train, y_train)
    # evaluation for best parameters
    precision = cross_val_score(grid_DT.best_estimator_, X_train, y_train, cv=3, scoring='precision')#预测为正类的样本中实际有多少是正
    recall = cross_val_score(grid_DT.best_estimator_, X_train, y_train, cv=3, scoring='recall')#实际是正类的样本有多少被预测出来
    fls = cross_val_score(grid_DT.best_estimator_, X_train, y_train, cv=3, scoring='f1')
    new = grid_DT.predict(df_.iloc[-1,1: ].values.reshape(1,-1))
    p_new = grid_DT.predict_proba(df_.iloc[-1,1: ].values.reshape(1,-1))[0][1]
    print('df_R'+ str(i), new, p_new, 'precison:', precision, 'recall:', recall, 'f1:', fls)
    print(grid_DT.best_estimator_)

endtime = datetime.now()
#end = time.time()
#print(start-end)
Time = endtime - starttime
print(Time.seconds, 'seconds')


Fitting 3 folds for each of 54 candidates, totalling 162 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 162 out of 162 | elapsed:    5.4s finished


df_R1 [0] 0.353598145612 precison: [ 0.20064725  0.23404255  0.20253165] recall: [ 0.45255474  0.40441176  0.47058824] f1: [ 0.27802691  0.29649596  0.28318584]
Pipeline(steps=[('clf', DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
            max_depth=20, max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=30,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=14, splitter='best'))])
Fitting 3 folds for each of 54 candidates, totalling 162 fits


[Parallel(n_jobs=-1)]: Done  47 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done 155 out of 162 | elapsed:    4.9s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done 162 out of 162 | elapsed:    5.0s finished


df_R2 [0] 0.347216578627 precison: [ 0.2244898   0.17843866  0.15289256] recall: [ 0.43650794  0.38095238  0.29365079] f1: [ 0.29649596  0.24303797  0.20108696]
Pipeline(steps=[('clf', DecisionTreeClassifier(class_weight='balanced', criterion='gini',
            max_depth=20, max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=20,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=14, splitter='best'))])
Fitting 3 folds for each of 54 candidates, totalling 162 fits


...

df_R31 [0] 0.276669071091 precison: [ 0.11764706  0.17889908  0.18461538] recall: [ 0.19834711  0.325       0.3       ] f1: [ 0.14769231  0.23076923  0.22857143]
Pipeline(steps=[('clf', DecisionTreeClassifier(class_weight='balanced', criterion='gini',
            max_depth=20, max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=30,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=14, splitter='best'))])
Fitting 3 folds for each of 54 candidates, totalling 162 fits


[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done 155 out of 162 | elapsed:    5.8s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done 162 out of 162 | elapsed:    5.8s finished


df_R32 [0] 0.394615845939 precison: [ 0.2125      0.19732441  0.20774648] recall: [ 0.38636364  0.4469697   0.45038168] f1: [ 0.27419355  0.2737819   0.28433735]
Pipeline(steps=[('clf', DecisionTreeClassifier(class_weight='balanced', criterion='gini',
            max_depth=20, max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=30,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=14, splitter='best'))])
Fitting 3 folds for each of 54 candidates, totalling 162 fits


[Parallel(n_jobs=-1)]: Done  44 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done 162 out of 162 | elapsed:    5.0s finished


df_R33 [0] 0.0 precison: [ 0.16326531  0.16371681  0.14068441] recall: [ 0.37383178  0.34579439  0.3490566 ] f1: [ 0.22727273  0.22222222  0.20054201]
Pipeline(steps=[('clf', DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
            max_depth=20, max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=20,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=14, splitter='best'))])
209 seconds


In [ ]:
# predict the last 10 times results
pro_ball = defaultdict(int)
pipeline_DT = Pipeline([('clf', DecisionTreeClassifier(random_state=14, criterion='gini'))])
paras_DT = {#'clf__max_depth': (2, 3, 4),
            #'clf__criterion': ('gini', 'entropy'),
            #'clf__min_samples_split': (2, 3),
            'clf__min_samples_leaf': (20, 30, 40)}
new =[]
for r in range(-10, 0):
    for i in range(1,34):
        df_ =  locals()['df_R'+ str(i)]
        X_train = df_.iloc[:r, : -1].values
        y_train = df_[99][:r].values
        y_train = [1 if x else 0 for x in y_train]
        grid_search_DT=GridSearchCV(pipeline_DT, paras_DT, n_jobs=-1, verbose=1)
        grid_search_DT.fit(X_train, y_train)
        pro_ball[i] = grid_search_DT.predict_proba(df_.iloc[r-1, 1: ].values.reshape(1, -1))
    
    new.append([k for (k, v2) in sorted({k: v2 for k, [[ v1,  v2]] in pro_ball.items()}.items(), key=itemgetter(1), reverse=True)[:6]])
    print(r)

endtime = datetime.now()
#end = time.time()
#print(start-end)
Time = endtime - starttime
print(Time.seconds, 'seconds')
new

In [ ]:
################    3.1 RF model parameters choose and predict for sorted data